In [1]:
#Import modules
import numpy as np
import pandas as pd
import os, sys
import time as t
import matplotlib as mpl
#mpl.use('Agg')
import pathlib
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.ticker import MaxNLocator

In [2]:
#CONSTANTS
cwd_PYTHON = os.getcwd() + '/'
PERIOD = 0.1
RSMALL = 0.15
RLARGE = 2.0*RSMALL
EPSILON = 0.4
AMPLITUDE = EPSILON*RLARGE
NX, NY, NZ = 126, 510, 126


DT = 1.0e-4
DENSITY = 2.0
MASS_LARGE = DENSITY*4.0*np.pi*RLARGE**3
MASS_SMALL = DENSITY*4.0*np.pi*RSMALL**3
OMEGA = 2.0*np.pi/PERIOD


SVAL_LIST = [0.5,1.0,2.0,3.0,4.0,5.0,7.5,10.0,50.0,100.0,150.0]
csfont = {'fontname':'Times New Roman'}

In [3]:
#GET FILE NAMES
# constructs a filepath for the force data of s = $sval
def fname(cwd,sval):
    strDir = cwd+'s{0}_h_nonlinear_v3.csv'.format(sval)
    return strDir

def plotName(cwd,sval,var,plane):
    #field = Vort, Pres, Vel, AvgW, AvgP, AvgU
    strDir = cwd+"../../Figures/eps0.4/NonLinear/v3/s{0}/".format(sval)
    pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
    return strDir+"nonlinear_s{0}_{1}_{2}.png".format(sval,var,plane)

def magName(cwd,method):
    strDir = cwd+"../../Figures/eps0.4/NonLinear/v3/"
    pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
    return strDir+"nonlinear_mags_vs_s2_{0}.png".format(method)

def magNameT(cwd,method):
    strDir = cwd+"../../Figures/eps0.4/NonLinear/v3/"
    pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
    return strDir+"nonlinear_mags_vs_s2_{0}_together.png".format(method)

# constructs a filepath for the force data of s = $sval
def posName(cwd,sval):
    strDir = cwd+'pd_e0.4_s{0}.txt'.format(sval)
    return strDir

In [4]:
def ExtractNonlinearTerms(sval,fieldData):
    global NX,NY,NZ
    #All field values to a list
    mxList = fieldData['mx'].values.tolist()
    myList = fieldData['my'].values.tolist()
    mzList = fieldData['mz'].values.tolist()
    
    oscCells = RemoveOscillationCells(sval,mxList,myList,mzList)
    fieldData.loc[oscCells,'fx'] = 0.0
    fieldData.loc[oscCells,'fy'] = 0.0
    fieldData.loc[oscCells,'fz'] = 0.0
    fieldData.loc[oscCells,'gx'] = 0.0
    fieldData.loc[oscCells,'gy'] = 0.0
    fieldData.loc[oscCells,'gz'] = 0.0
    fieldData.loc[oscCells,'hx'] = 0.0
    fieldData.loc[oscCells,'hy'] = 0.0
    fieldData.loc[oscCells,'hz'] = 0.0
    fieldData.loc[oscCells,'lapux'] = 0.0
    fieldData.loc[oscCells,'lapuy'] = 0.0
    fieldData.loc[oscCells,'lapuz'] = 0.0
    fieldData.loc[oscCells,'gradpx'] = 0.0
    fieldData.loc[oscCells,'gradpy'] = 0.0
    fieldData.loc[oscCells,'gradpz'] = 0.0
    fieldData.loc[oscCells,'g2x'] = 0.0
    fieldData.loc[oscCells,'g2y'] = 0.0
    fieldData.loc[oscCells,'g2z'] = 0.0
    fieldData.loc[oscCells,'h2x'] = 0.0
    fieldData.loc[oscCells,'h2y'] = 0.0
    fieldData.loc[oscCells,'h2z'] = 0.0
    
    fxList = fieldData['fx'].values.tolist()
    fyList = fieldData['fy'].values.tolist()
    fzList = fieldData['fz'].values.tolist()
    gxList = fieldData['gx'].values.tolist()
    gyList = fieldData['gy'].values.tolist()
    gzList = fieldData['gz'].values.tolist()
    hxList = fieldData['hx'].values.tolist()
    hyList = fieldData['hy'].values.tolist()
    hzList = fieldData['hz'].values.tolist()
    lapuxList = fieldData['lapux'].values.tolist()
    lapuyList = fieldData['lapuy'].values.tolist()
    lapuzList = fieldData['lapuz'].values.tolist()
    gradpxList = fieldData['gradpx'].values.tolist()
    gradpyList = fieldData['gradpy'].values.tolist()
    gradpzList = fieldData['gradpz'].values.tolist()
    g2xList = fieldData['g2x'].values.tolist()
    g2yList = fieldData['g2y'].values.tolist()
    g2zList = fieldData['g2z'].values.tolist()
    h2xList = fieldData['h2x'].values.tolist()
    h2yList = fieldData['h2y'].values.tolist()
    h2zList = fieldData['h2z'].values.tolist()
    #Convert lists to numpy arrays
    #Reshape them to be NX x NY X NZ
    mx = np.array(mxList).reshape((NX,NY,NZ))
    my = np.array(myList).reshape((NX,NY,NZ))
    mz = np.array(mzList).reshape((NX,NY,NZ))
    fx = np.array(fxList).reshape((NX,NY,NZ))
    fy = np.array(fyList).reshape((NX,NY,NZ))
    fz = np.array(fzList).reshape((NX,NY,NZ))
    gx = np.array(gxList).reshape((NX,NY,NZ))
    gy = np.array(gyList).reshape((NX,NY,NZ))
    gz = np.array(gzList).reshape((NX,NY,NZ))
    hx = np.array(hxList).reshape((NX,NY,NZ))
    hy = np.array(hyList).reshape((NX,NY,NZ))
    hz = np.array(hzList).reshape((NX,NY,NZ))
    lapux = np.array(lapuxList).reshape((NX,NY,NZ))
    lapuy = np.array(lapuyList).reshape((NX,NY,NZ))
    lapuz = np.array(lapuzList).reshape((NX,NY,NZ))
    gradpx = np.array(gradpxList).reshape((NX,NY,NZ))
    gradpy = np.array(gradpyList).reshape((NX,NY,NZ))
    gradpz = np.array(gradpzList).reshape((NX,NY,NZ))
    g2x = np.array(g2xList).reshape((NX,NY,NZ))
    g2y = np.array(g2yList).reshape((NX,NY,NZ))
    g2z = np.array(g2zList).reshape((NX,NY,NZ))
    h2x = np.array(h2xList).reshape((NX,NY,NZ))
    h2y = np.array(h2yList).reshape((NX,NY,NZ))
    h2z = np.array(h2zList).reshape((NX,NY,NZ))
    
    return (mx,my,mz,fx,fy,fz,gx,gy,gz,hx,hy,hz,lapux,lapuy,lapuz,gradpx,gradpy,gradpz,g2x,g2y,g2z,h2x,h2y,h2z)

def SubsetZYPlane(my,mz,xvar,yvar,zvar):
    global NX
    mym = my[int(0.5*NX-1),:,:].copy()
    myp = my[int(0.5*NX),:,:].copy()
    my0 = 0.5*(mym+myp)
    mzm = mz[int(0.5*NX-1),:,:].copy()
    mzp = mz[int(0.5*NX),:,:].copy()
    mz0 = 0.5*(mzm+mzp)
    xvarm = xvar[int(0.5*NX-1),:,:].copy()
    xvarp = xvar[int(0.5*NX),:,:].copy()
    xvar0 = 0.5*(xvarm+xvarp)
    yvarm = yvar[int(0.5*NX-1),:,:].copy()
    yvarp = yvar[int(0.5*NX),:,:].copy()
    yvar0 = 0.5*(yvarm+yvarp)
    zvarm = zvar[int(0.5*NX-1),:,:].copy()
    zvarp = zvar[int(0.5*NX),:,:].copy()
    zvar0 = 0.5*(zvarm+zvarp)
    
    return (my0.T,mz0.T,xvar0.T,yvar0.T,zvar0.T)

def SubsetXYPlane(mx,my,xvar,yvar,zvar):
    global NZ
    mxm = mx[:,:,int(0.5*NZ-1)].copy()
    mxp = mx[:,:,int(0.5*NZ)].copy()
    mx0 = 0.5*(mxm+mxp)
    mym = my[:,:,int(0.5*NZ-1)].copy()
    myp = my[:,:,int(0.5*NZ)].copy()
    my0 = 0.5*(mym+myp)
    xvarm = xvar[:,:,int(0.5*NZ-1)].copy()
    xvarp = xvar[:,:,int(0.5*NZ)].copy()
    xvar0 = 0.5*(xvarm+xvarp)
    yvarm = yvar[:,:,int(0.5*NZ-1)].copy()
    yvarp = yvar[:,:,int(0.5*NZ)].copy()
    yvar0 = 0.5*(yvarm+yvarp)
    zvarm = zvar[:,:,int(0.5*NZ-1)].copy()
    zvarp = zvar[:,:,int(0.5*NZ)].copy()
    zvar0 = 0.5*(zvarm+zvarp)
    
    return (mx0,my0,xvar0,yvar0,zvar0)

def SubsetXZPlane(mx,my,xvar,yvar,zvar):
    global NY
    mxm = mx[:,int(0.5*NY-1),:].copy()
    mxp = mx[:,int(0.5*NY),:].copy()
    mx0 = 0.5*(mxm+mxp)
    mym = my[:,int(0.5*NY-1),:].copy()
    myp = my[:,int(0.5*NY),:].copy()
    my0 = 0.5*(mym+myp)
    xvarm = xvar[:,int(0.5*NY-1),:].copy()
    xvarp = xvar[:,int(0.5*NY),:].copy()
    xvar0 = 0.5*(xvarm+xvarp)
    yvarm = yvar[:,int(0.5*NY-1),:].copy()
    yvarp = yvar[:,int(0.5*NY),:].copy()
    yvar0 = 0.5*(yvarm+yvarp)
    zvarm = zvar[:,int(0.5*NY-1),:].copy()
    zvarp = zvar[:,int(0.5*NY),:].copy()
    zvar0 = 0.5*(zvarm+zvarp)
    
    return (mx0,my0,xvar0,yvar0,zvar0)

In [14]:
def PlotNonlinearField(mx,my,xvar,yvar,zvar,pars):
    global cwd_PYTHON
    par = pars[0]
    field = pars[1]
    plane = pars[2]
    if(plane == 'XY' or plane == 'ZY'):
        fig, ax = plt.subplots(nrows=1, ncols=3,figsize=(13,16),dpi=200)
        ax[0].axis([-1.0,1.0,-4.0,4.0])
        ax[1].axis([-1.0,1.0,-4.0,4.0])
        ax[2].axis([-1.0,1.0,-4.0,4.0])
    else:
        fig, ax = plt.subplots(nrows=1, ncols=3,figsize=(13,4),dpi=200)
        ax[0].axis([-1.0,1.0,-1.0,1.0])
        ax[1].axis([-1.0,1.0,-1.0,1.0])
        ax[2].axis([-1.0,1.0,-1.0,1.0])
    
    ax[0].set_title(r'%s Plane: $s^2$ = %.1f: f%s'%(plane,par,field),fontsize=16)
    ax[1].set_title(r'$s^2$ = %.1f: g%s'%(par,field),fontsize=16)
    ax[2].set_title(r'$s^2$ = %.1f: h%s'%(par,field),fontsize=16)
    
    #levels = MaxNLocator(nbins=21).tick_values(xvar.min()/2.0, xvar.max()/2.0)
    levels = MaxNLocator(nbins=21).tick_values(-50.0, 50.0)
    cf0 = ax[0].contourf(mx,my,xvar,cmap='viridis',levels=levels,extend='both')
    fig.colorbar(cf0,ax=ax[0],shrink=0.75)
    #levels = MaxNLocator(nbins=21).tick_values(yvar.min()/2.0, yvar.max()/2.0)
    levels = MaxNLocator(nbins=21).tick_values(-50.0, 50.0)
    cf1 = ax[1].contourf(mx,my,yvar,cmap='viridis',levels=levels,extend='both')
    fig.colorbar(cf1,ax=ax[1],shrink=0.75)
    #levels = MaxNLocator(nbins=21).tick_values(zvar.min()/2.0, zvar.max()/2.0)
    levels = MaxNLocator(nbins=21).tick_values(-50.0, 50.0)
    cf2 = ax[2].contourf(mx,my,zvar,cmap='viridis',levels=levels,extend='both')
    fig.colorbar(cf2,ax=ax[2],shrink=0.75)
    
    ax[0].set_aspect('equal')
    ax[1].set_aspect('equal')
    ax[2].set_aspect('equal')
    fig.tight_layout()
    fig.savefig(plotName(cwd_PYTHON,par,field,plane))
    fig.clf()
    plt.close()
    return

def PlotNonlinear_vs_s2(s2,sum_values,method):
    global NX,NY,NZ
    f_values = sum_values[:,0]
    g_values = sum_values[:,1]
    h_values = sum_values[:,2]
    
    
    fig, ax = plt.subplots(nrows=1, ncols=3,figsize=(13,3),dpi=200)
    ax[0].set_title(r'$\langle f \rangle$ vs. $s^2$',fontsize=16,**csfont)
    ax[1].set_title(r'$\langle g \rangle$ vs. $s^2$',fontsize=16,**csfont)
    ax[2].set_title(r'$\langle h \rangle$ vs. $s^2$',fontsize=16,**csfont)
    ax[0].plot(s2,f_values/(NX*NY*NZ),c='tab:red',label='f')
    ax[0].scatter(s2,f_values/(NX*NY*NZ),c='tab:red',s=12)
    ax[1].plot(s2,g_values/(NX*NY*NZ),c='tab:blue',label='g')
    ax[1].scatter(s2,g_values/(NX*NY*NZ),c='tab:blue',s=12)
    ax[2].plot(s2,h_values/(NX*NY*NZ),c='tab:green',label='h')
    ax[2].scatter(s2,h_values/(NX*NY*NZ),c='tab:green',s=12)
    
    ax[0].set_xlim([0.0,175.0])
    ax[1].set_xlim([0.0,175.0])
    ax[2].set_xlim([0.0,175.0])
    fig.tight_layout()
    fig.savefig(magName(cwd_PYTHON,method))
    fig.clf()
    plt.close()
    return

def PlotNonlinear_vs_s2_together(s2,sum_values,method):
    f_values = sum_values[:,0]
    g_values = sum_values[:,1]
    h_values = sum_values[:,2]
    
    
    fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(8,6),dpi=200)
    ax.set_title(r'$\langle f,g,h \rangle$ vs. $s^2$',fontsize=16,**csfont)
    ax.set_xlabel(r'$s^2$',fontsize=12,**csfont)
    ax.set_ylabel(r'$\langle f,g,h \rangle$',fontsize=12,**csfont)
    ax.plot(s2,f_values/(NX*NY*NZ),c='tab:red',label='f')
    ax.scatter(s2,f_values/(NX*NY*NZ),c='tab:red',s=12)
    ax.plot(s2,g_values/(NX*NY*NZ),c='tab:blue',label='g')
    ax.scatter(s2,g_values/(NX*NY*NZ),c='tab:blue',s=12)
    ax.plot(s2,h_values/(NX*NY*NZ),c='tab:green',label='h')
    ax.scatter(s2,h_values/(NX*NY*NZ),c='tab:green',s=12)
    
    ax.legend(loc='best',fontsize='x-small')
    ax.set_xlim([0.0,175.0])
    fig.tight_layout()
    fig.savefig(magNameT(cwd_PYTHON,method))
    fig.clf()
    plt.close()
    return

In [11]:
def GetDistance(x0,x1):
    diff = x1 - x0
    return np.sqrt(diff[0]**2 + diff[1]**2 + diff[2]**2)

def GetRelevantPosData(cwd,sval):
    data = np.loadtxt(posName(cwd,sval),delimiter=' ')
    dataDict = {'xU':data[:,0],'yU':data[:,1],
                'xL':data[:,2],'yL':data[:,3],
                'curr':data[:,4],'des':data[:,5],'time':data[:,6]}
    posData = pd.DataFrame(data=dataDict)
    posData = posData.sort_values(by=['time'])
    posData = posData.reset_index(drop=True)
    nTime = len(posData['time'])
    print('nTime = ',nTime)
    nPer = int(np.trunc(1.0*nTime*(DT/PERIOD)))
    nPer = nPer - 1
    print('nPer = ',nPer)
    startIdx, endIdx = int((nPer-1)*(PERIOD/DT)), int(nPer*(PERIOD/DT))
    print('startIdx = ',startIdx)
    print('endIdx = ',endIdx)
    #Subset to last full oscillation
    oscData = posData[startIdx:endIdx]
    print('nTime = ',len(oscData['time']))
    oscData = oscData.reset_index(drop=True)
    #Find when small sphere moves more than DX
    DX = 2.0/128.0
    lastPos = np.array([oscData.loc[0,'xL'], oscData.loc[0,'yL'],0.0])
    indices = list()
    #indices = np.array([])
    for idx in range(len(oscData['time'])):
        currPos = np.array([oscData.loc[idx,'xL'], oscData.loc[idx,'yL'],0.0])
        dist = GetDistance(lastPos,currPos)
        if(dist >= DX):
            #Record index value in list
            #Make currPos the new lastPos
            indices.append(idx)#np.append(indices,idx)
            lastPos = currPos
    print('indices = ',indices)
    relData = oscData.iloc[indices].copy()
    print(relData)
    relData = relData.reset_index(drop=True)
            
    return relData

def RemoveOscillationCells(sval,mx,my,mz):
    global RSMALL,RLARGE
    cwd_POS = cwd_PYTHON+'../../PosData/eps0.4/'
    posData = GetRelevantPosData(cwd_POS,sval)
    #Given relevant pos data, for each time find the cells which are inside the sphere's location
    oscillationCells = list()
    #For each relevant time step
    for idx in range(len(posData['time'])):
        print('time = ',posData.loc[idx,'time'])
        nCells = len(mx)
        xU = [posData.loc[idx,'xU']]*nCells
        yU = [posData.loc[idx,'yU']]*nCells
        xL = [posData.loc[idx,'xL']]*nCells
        yL = [posData.loc[idx,'yL']]*nCells
        distDict = {'xU':xU,'yU':yU,'xL':xL,'yL':yL,'mx':mx,'my':my,'mz':mz}
        distData = pd.DataFrame(data=distDict)
        CM_Large = np.array([distData['xU'],distData['yU'],[0.0 for a in range(nCells)]])
        CM_Small = np.array([distData['xL'],distData['yL'],[0.0 for a in range(nCells)]])
        meshLoc  = np.array([distData['mx'],distData['my'],distData['mz']])
        distData['distL'] = GetDistance(CM_Large,meshLoc)
        distData['distS'] = GetDistance(CM_Small,meshLoc)
        largeData = distData[distData['distL'] <= RLARGE].copy()
        indicesL = largeData.index.values.tolist()
        for jdx in range(len(indicesL)):
            oscillationCells.append(indicesL[jdx])
        smallData = distData[distData['distS'] <= RSMALL].copy()
        indicesS = smallData.index.values.tolist()
        for jdx in range(len(indicesS)):
            oscillationCells.append(indicesS[jdx])
        print('n_oscillationCells = ',len(oscillationCells))
    oscCells = list(dict.fromkeys(oscillationCells))
    #oscCells = list(set(oscillationCells)).sort()
    print('n_oscCells = ',len(oscCells))
    return oscCells

def RemoveOscillationCells_Slow(sval,mx,my):
    global RSMALL,RLARGE
    cwd_POS = cwd_PYTHON+'../../PosData/eps0.4/'
    posData = GetRelevantPosData(cwd_POS,sval)
    #Given relevant pos data, for each time find the cells which are inside the sphere's location
    oscillationCells = list()
    #For each relevant time step
    for idx in range(len(posData['time'])):
        print('time = ',posData.loc[idx,'time'])
        CM_Large = np.array([posData.loc[idx,'xU'],posData.loc[idx,'yU']])
        CM_Small = np.array([posData.loc[idx,'xL'],posData.loc[idx,'yL']])
        nCells = len(mx)
        print('nCells = ',nCells)
        for jdx in range(nCells):
            if(jdx % 1000000 == 0):
                print('jdx = ',jdx)
            meshLoc  = np.array([mx[jdx],my[jdx]])
            distL = GetDistance(CM_Large,meshLoc)
            distS = GetDistance(CM_Small,meshLoc)
            if((distL <= RLARGE) or (distS <= RSMALL)):
                #mesh cell is inside oscillation path: Record
                oscillationCells.append(jdx)
        print('n_oscillationCells = ',len(oscillationCells))
    oscCells = list(dict.fromkeys(oscillationCells))
    #oscCells = list(set(oscillationCells)).sort()
    print('n_oscCells = ',len(oscCells))
    return oscCells

#cwd_DATA = '/Volumes/Seagate Backup Plus Drive/Projects/3D/Felderhof/NonlinearData/eps0.4/v2/'
#simData = pd.read_csv(fname(cwd_DATA,2.0),delimiter=' ')
#print('Data for s^2 = %.1f has been acquired'%(2.0))
#Extract f,g, and h values (x,y,z) and format to be a 3D matrix
#mx,my,mz,fx,fy,fz,gx,gy,gz,hx,hy,hz,lapux,lapuy,lapuz,gradpx,gradpy,gradpz = ExtractNonlinearTerms(simData)
#oscCells = RemoveOscillationCells(2.0,mx.flatten(),my.flatten())
    

In [15]:
#Plan of Attack
#1) For each sval, obtain nonlinear data
#2) Get sum of f,g,h values in array
#3) Plot mid planes of f,g,h values (x-y, y-z, x-z)
#4) Plot sum(h) vs s^2

sum_values = np.zeros((len(SVAL_LIST),3))#f = 0; g = 1; h = 2;
sum2_values = np.zeros((len(SVAL_LIST),3))#f = 0; g2 = 1; h2 = 2; 

#cwd_DATA = '/Volumes/Seagate Backup Plus Drive/Projects/3D/Felderhof/NonlinearData/eps0.4/'
cwd_DATA = '/Volumes/Seagate Backup Plus Drive/Projects/3D/Felderhof/NonlinearData/eps0.4/v3/'

RHO = 2.0
OMEGA = 2.0*np.pi*10.0
RADIUSLARGE = 0.3

count = 0
for sval in SVAL_LIST:
    simData = pd.read_csv(fname(cwd_DATA,sval),delimiter=' ')
    print('Data for s^2 = %.1f has been acquired'%(sval))
    #Extract f,g, and h values (x,y,z) and format to be a 3D matrix
    mx,my,mz,fx,fy,fz,gx,gy,gz,hx,hy,hz,lapux,lapuy,lapuz,gradpx,gradpy,gradpz,g2x,g2y,g2z,h2x,h2y,h2z = ExtractNonlinearTerms(sval,simData)
    
    MU1 = 2.0*RHO*OMEGA*RADIUSLARGE**2/float(sval)
    print('bad MU? = ',MU1)
    MU = 2.0*RHO*OMEGA*RADIUSLARGE**(2)/float(sval)
    print('Mu = ',MU)
    lapux, lapuy, lapuz = lapux*MU, lapuy*MU, lapuz*MU
    #Subset data into XY, ZY, and XZ Planes
    #XY
    pars = [sval,'f','XY']
    mx_z0,my_z0,fx_z0,fy_z0,fz_z0 = SubsetXYPlane(mx,my,fx,fy,fz)
    #PlotNonlinearField(mx_z0,my_z0,fx_z0,fy_z0,fz_z0,pars)
    #pars[1] = 'g'
    mx_z0,my_z0,gx_z0,gy_z0,gz_z0 = SubsetXYPlane(mx,my,gx,gy,gz)
    #PlotNonlinearField(mx_z0,my_z0,gx_z0,gy_z0,gz_z0,pars)
    #pars[1] = 'h'
    mx_z0,my_z0,hx_z0,hy_z0,hz_z0 = SubsetXYPlane(mx,my,hx,hy,hz)
    #PlotNonlinearField(mx_z0,my_z0,hx_z0,hy_z0,hz_z0,pars)
    #pars[1] = 'lapu'
    #mx_z0,my_z0,lapux_z0,lapuy_z0,lapuz_z0 = SubsetXYPlane(mx,my,lapux,lapuy,lapuz)
    #PlotNonlinearField(mx_z0,my_z0,lapux_z0,lapuy_z0,lapuz_z0,pars)
    #pars[1] = 'gradp'
    #mx_z0,my_z0,gradpx_z0,gradpy_z0,gradpz_z0 = SubsetXYPlane(mx,my,gradpx,gradpy,gradpz)
    #PlotNonlinearField(mx_z0,my_z0,gradpx_z0,gradpy_z0,gradpz_z0,pars)
    #pars[1] = 'g2'
    mx_z0,my_z0,g2x_z0,g2y_z0,g2z_z0 = SubsetXYPlane(mx,my,g2x,g2y,g2z)
    #PlotNonlinearField(mx_z0,my_z0,g2x_z0,g2y_z0,g2z_z0,pars)
    #pars[1] = 'h2'
    mx_z0,my_z0,h2x_z0,h2y_z0,h2z_z0 = SubsetXYPlane(mx,my,h2x,h2y,h2z)
    #PlotNonlinearField(mx_z0,my_z0,h2x_z0,h2y_z0,h2z_z0,pars)
    
    pars[1] = 'x'
    PlotNonlinearField(mx_z0,my_z0,fx_z0,gx_z0,hx_z0,pars)
    pars[1] = 'y'
    PlotNonlinearField(mx_z0,my_z0,fy_z0,gy_z0,hy_z0,pars)
    pars[1] = 'z'
    PlotNonlinearField(mx_z0,my_z0,fz_z0,gz_z0,hz_z0,pars)
    pars[1] = 'x2'
    PlotNonlinearField(mx_z0,my_z0,fx_z0,g2x_z0,h2x_z0,pars)
    pars[1] = 'y2'
    PlotNonlinearField(mx_z0,my_z0,fy_z0,g2y_z0,h2y_z0,pars)
    pars[1] = 'z2'
    PlotNonlinearField(mx_z0,my_z0,fz_z0,g2z_z0,h2z_z0,pars)
    
    print('s2 = %.1f: XY subplanes are plotted!'%(sval))
    #ZY
    pars = [sval,'f','ZY']
    mz_x0,my_x0,fx_x0,fy_x0,fz_x0 = SubsetZYPlane(mz,my,fx,fy,fz)
    #PlotNonlinearField(mz_x0,my_x0,fx_x0,fy_x0,fz_x0,pars)
    #pars[1] = 'g'
    mz_x0,my_x0,gx_x0,gy_x0,gz_x0 = SubsetZYPlane(mz,my,gx,gy,gz)
    #PlotNonlinearField(mz_x0,my_x0,gx_x0,gy_x0,gz_x0,pars)
    #pars[1] = 'h'
    mz_x0,my_x0,hx_x0,hy_x0,hz_x0 = SubsetZYPlane(mz,my,hx,hy,hz)
    #PlotNonlinearField(mz_x0,my_x0,hx_x0,hy_x0,hz_x0,pars)
    #pars[1] = 'lapu'
    #mz_x0,my_x0,lapux_x0,lapuy_x0,lapuz_x0 = SubsetZYPlane(mz,my,lapux,lapuy,lapuz)
    #PlotNonlinearField(mz_x0,my_x0,lapux_x0,lapuy_x0,lapuz_x0,pars)
    #pars[1] = 'gradp'
    #mz_x0,my_x0,gradpx_x0,gradpy_x0,gradpz_x0 = SubsetZYPlane(mz,my,gradpx,gradpy,gradpz)
    #PlotNonlinearField(mz_x0,my_x0,gradpx_x0,gradpy_x0,gradpz_x0,pars)
    #pars[1] = 'g2'
    mz_x0,my_x0,g2x_x0,g2y_x0,g2z_x0 = SubsetZYPlane(mz,my,g2x,g2y,g2z)
    #PlotNonlinearField(mz_x0,my_x0,g2x_x0,g2y_x0,g2z_x0,pars)
    #pars[1] = 'h2'
    mz_x0,my_x0,h2x_x0,h2y_x0,h2z_x0 = SubsetZYPlane(mz,my,h2x,h2y,h2z)
    #PlotNonlinearField(mz_x0,my_x0,h2x_x0,h2y_x0,h2z_x0,pars)
    
    pars[1] = 'x'
    PlotNonlinearField(mz_x0,my_x0,fx_x0,gx_x0,hx_x0,pars)
    pars[1] = 'y'
    PlotNonlinearField(mz_x0,my_x0,fy_x0,gy_x0,hy_x0,pars)
    pars[1] = 'z'
    PlotNonlinearField(mz_x0,my_x0,fz_x0,gz_x0,hz_x0,pars)
    pars[1] = 'x2'
    PlotNonlinearField(mz_x0,my_x0,fx_x0,g2x_x0,h2x_x0,pars)
    pars[1] = 'y2'
    PlotNonlinearField(mz_x0,my_x0,fy_x0,g2y_x0,h2y_x0,pars)
    pars[1] = 'z2'
    PlotNonlinearField(mz_x0,my_x0,fz_x0,g2z_x0,h2z_x0,pars)
    
    print('s2 = %.1f: ZY subplanes are plotted!'%(sval))
    #XZ
    pars = [sval,'f','XZ']
    mx_y0,mz_y0,fx_y0,fy_y0,fz_y0 = SubsetXZPlane(mx,mz,fx,fy,fz)
    #PlotNonlinearField(mx_y0,mz_y0,fx_y0,fy_y0,fz_y0,pars)
    #pars[1] = 'g'
    mx_y0,mz_y0,gx_y0,gy_y0,gz_y0 = SubsetXZPlane(mx,mz,gx,gy,gz)
    #PlotNonlinearField(mx_y0,mz_y0,gx_y0,gy_y0,gz_y0,pars)
    #pars[1] = 'h'
    mx_y0,mz_y0,hx_y0,hy_y0,hz_y0 = SubsetXZPlane(mx,mz,hx,hy,hz)
    #PlotNonlinearField(mx_y0,mz_y0,hx_y0,hy_y0,hz_y0,pars)
    #pars[1] = 'lapu'
    #mx_y0,mz_y0,lapux_y0,lapuy_y0,lapuz_y0 = SubsetXZPlane(mx,mz,lapux,lapuy,lapuz)
    #PlotNonlinearField(mx_y0,mz_y0,lapux_y0,lapuy_y0,lapuz_y0,pars)
    #pars[1] = 'gradp'
    #mx_y0,mz_y0,gradpx_y0,gradpy_y0,gradpz_y0 = SubsetXZPlane(mx,mz,gradpx,gradpy,gradpz)
    #PlotNonlinearField(mx_y0,mz_y0,gradpx_y0,gradpy_y0,gradpz_y0,pars)
    #pars[1] = 'g2'
    mx_y0,mz_y0,g2x_y0,g2y_y0,g2z_y0 = SubsetXZPlane(mx,mz,g2x,g2y,g2z)
    #PlotNonlinearField(mx_y0,mz_y0,g2x_y0,g2y_y0,g2z_y0,pars)
    #pars[1] = 'h2'
    mx_y0,mz_y0,h2x_y0,h2y_y0,h2z_y0 = SubsetXZPlane(mx,mz,h2x,h2y,h2z)
    #PlotNonlinearField(mx_y0,mz_y0,h2x_y0,h2y_y0,h2z_y0,pars)
    
    pars[1] = 'x'
    PlotNonlinearField(mx_y0,mz_y0,fx_y0,gx_y0,hx_y0,pars)
    pars[1] = 'y'
    PlotNonlinearField(mx_y0,mz_y0,fy_y0,gy_y0,hy_y0,pars)
    pars[1] = 'z'
    PlotNonlinearField(mx_y0,mz_y0,fz_y0,gz_y0,hz_y0,pars)
    pars[1] = 'x2'
    PlotNonlinearField(mx_y0,mz_y0,fx_y0,g2x_y0,h2x_y0,pars)
    pars[1] = 'y2'
    PlotNonlinearField(mx_y0,mz_y0,fy_y0,g2y_y0,h2y_y0,pars)
    pars[1] = 'z2'
    PlotNonlinearField(mx_y0,mz_y0,fz_y0,g2z_y0,h2z_y0,pars)
    
    print('s2 = %.1f: XZ subplanes are plotted!'%(sval))
    #Obtain sum of f,g,h values 
    fmag = np.sqrt(fx*fx + fy*fy + fz*fz)
    gmag = np.sqrt(gx*gx + gy*gy + gz*gz)
    hmag = np.sqrt(hx*hx + hy*hy + hz*hz)
    g2mag = np.sqrt(g2x*g2x + g2y*g2y + g2z*g2z)
    h2mag = np.sqrt(h2x*h2x + h2y*h2y + h2z*h2z)
    sum_values[count,0] = np.sum(fmag.flatten())
    sum_values[count,1] = np.sum(gmag.flatten())
    sum_values[count,2] = np.sum(hmag.flatten())
    sum2_values[count,0] = np.sum(fmag.flatten())
    sum2_values[count,1] = np.sum(g2mag.flatten())
    sum2_values[count,2] = np.sum(h2mag.flatten())
    count += 1


Data for s^2 = 0.5 has been acquired
nTime =  2498
nPer =  1
startIdx =  0
endIdx =  1000
nTime =  1000
indices =  [47, 71, 96, 126, 167, 259, 355, 396, 429, 459, 487, 514, 541, 569, 599, 633, 676, 892, 926, 956, 984]
               xU        yU        xL        yL      curr       des    time
47  -1.140290e-03  0.453664 -0.000238 -0.463718  0.916615  0.935645  0.0048
71  -8.064050e-04  0.457857  0.000131 -0.479613  0.936606  0.952454  0.0072
96  -4.738040e-04  0.462530  0.000201 -0.495741  0.957487  0.968692  0.0097
126 -1.452210e-04  0.467563  0.000109 -0.511703  0.978646  0.985912  0.0127
167  1.379150e-04  0.472942 -0.000003 -0.527577  1.000090  1.004420  0.0168
259  2.156190e-04  0.479044  0.000003 -0.543210  1.022220  1.019760  0.0260
355  4.494690e-05  0.475555  0.000014 -0.527534  1.003500  0.994355  0.0356
396 -2.859070e-08  0.471143  0.000013 -0.511758  0.983464  0.972352  0.0397
429 -1.843340e-05  0.466588  0.000012 -0.496047  0.963293  0.951094  0.0430
459 -2.427280e-05  0.4

n_oscillationCells =  32720
time =  3.3054
n_oscillationCells =  65524
time =  3.3083
n_oscillationCells =  98324
time =  3.3115
n_oscillationCells =  131076
time =  3.3155
n_oscillationCells =  163832
time =  3.324
n_oscillationCells =  196620
time =  3.3339
n_oscillationCells =  229420
time =  3.3379
n_oscillationCells =  262152
time =  3.3412
n_oscillationCells =  294900
time =  3.3441
n_oscillationCells =  327624
time =  3.3468
n_oscillationCells =  360376
time =  3.3494
n_oscillationCells =  393112
time =  3.352
n_oscillationCells =  425840
time =  3.3547
n_oscillationCells =  458604
time =  3.3575
n_oscillationCells =  491392
time =  3.3606
n_oscillationCells =  524176
time =  3.3643
n_oscillationCells =  556900
time =  3.3699
n_oscillationCells =  589696
time =  3.3851
n_oscillationCells =  622492
time =  3.3888
n_oscillationCells =  655228
time =  3.3919
n_oscillationCells =  687968
time =  3.3947
n_oscillationCells =  720716
time =  3.3974
n_oscillationCells =  753452
time =  

s2 = 4.0: XY subplanes are plotted!
s2 = 4.0: ZY subplanes are plotted!
s2 = 4.0: XZ subplanes are plotted!
Data for s^2 = 5.0 has been acquired
nTime =  41426
nPer =  40
startIdx =  39000
endIdx =  40000
nTime =  1000
indices =  [25, 51, 79, 110, 148, 214, 339, 377, 408, 436, 462, 488, 513, 539, 566, 596, 631, 679, 855, 890, 920, 947, 973, 998]
               xU        yU            xL        yL      curr       des  \
25   2.470870e-07  0.409539  6.023030e-07 -0.509138  0.917919  0.919516   
51   2.698640e-07  0.413163  5.441270e-07 -0.524869  0.937303  0.938513   
79   2.691960e-07  0.416997  7.273760e-07 -0.540730  0.957051  0.957810   
110  2.295890e-07  0.421019  9.255660e-07 -0.556410  0.976836  0.977070   
148  1.519250e-07  0.425408  6.457020e-07 -0.572109  0.997055  0.996637   
214 -3.301870e-09  0.430919  9.285010e-07 -0.587836  1.018580  1.017110   
339 -1.090290e-07  0.431533  2.470470e-06 -0.572038  1.003970  1.001320   
377 -4.845460e-08  0.429059  2.982430e-06 -0.556352 

n_oscillationCells =  32752
time =  4.8052
n_oscillationCells =  65476
time =  4.808
n_oscillationCells =  98244
time =  4.8111
n_oscillationCells =  130996
time =  4.8149
n_oscillationCells =  163716
time =  4.8212
n_oscillationCells =  196416
time =  4.834
n_oscillationCells =  229132
time =  4.8378
n_oscillationCells =  261876
time =  4.8409
n_oscillationCells =  294620
time =  4.8437
n_oscillationCells =  327432
time =  4.8463
n_oscillationCells =  360168
time =  4.8488
n_oscillationCells =  392932
time =  4.8513
n_oscillationCells =  425660
time =  4.8538
n_oscillationCells =  458436
time =  4.8565
n_oscillationCells =  491232
time =  4.8594
n_oscillationCells =  523968
time =  4.8627
n_oscillationCells =  556672
time =  4.8671
n_oscillationCells =  589460
time =  4.8861
n_oscillationCells =  622200
time =  4.8894
n_oscillationCells =  654968
time =  4.8923
n_oscillationCells =  687664
time =  4.895
n_oscillationCells =  720428
time =  4.8975
n_oscillationCells =  753172
time =  4

s2 = 100.0: XY subplanes are plotted!
s2 = 100.0: ZY subplanes are plotted!
s2 = 100.0: XZ subplanes are plotted!
Data for s^2 = 150.0 has been acquired
nTime =  50000
nPer =  49
startIdx =  48000
endIdx =  49000
nTime =  1000
indices =  [24, 49, 75, 104, 138, 185, 354, 388, 417, 443, 468, 492, 516, 541, 567, 595, 627, 668, 866, 898, 926, 952, 977]
               xU        yU            xL        yL      curr       des  \
24   3.299970e-07  0.480799  2.231120e-06 -0.438679  0.918726  0.918772   
49   2.840670e-07  0.483291  1.951020e-06 -0.454648  0.937215  0.937082   
75   2.472080e-07  0.485790  1.642160e-06 -0.470351  0.955465  0.955150   
104  2.175460e-07  0.488386  1.522750e-06 -0.486275  0.974060  0.973549   
138  1.853750e-07  0.491059  1.088750e-06 -0.502125  0.992696  0.991979   
185  1.074280e-07  0.493884  1.244290e-07 -0.517787  1.011370  1.010430   
354 -3.567620e-07  0.493270 -2.008470e-06 -0.502107  0.995844  0.994819   
388 -3.677930e-07  0.491142 -2.131390e-06 -0.4862

In [9]:
s2_values = np.array(SVAL_LIST)
PlotNonlinear_vs_s2(s2_values,sum_values,'first')
PlotNonlinear_vs_s2_together(s2_values,sum_values,'first')
PlotNonlinear_vs_s2(s2_values,sum2_values,'second')
PlotNonlinear_vs_s2_together(s2_values,sum2_values,'second')